In [2]:
import os
print('hi')
import requests
import pandas as pd
from sqlalchemy import create_engine, text
from tqdm import tqdm
from jsonschema import validate
import json


hi


In [4]:
DB_USER = os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")
DB_HOST = os.getenv("DB_HOST")
DB_NAME = os.getenv("DB_NAME")
DB_PORT = os.getenv("DB_PORT", 5432)

# --- Connect to PostgreSQL ---
conn_str = f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(conn_str)

# --- Create table schema if not exists ---



create_table_sql = """
DROP TABLE IF EXISTS drug_properties;

CREATE TABLE IF NOT EXISTS drug_properties (
    chembl_id TEXT,
    smiles TEXT,
    binding_free_energy FLOAT,
    solubility FLOAT,
    logp FLOAT,
    permeability FLOAT,
    pka FLOAT,
    source TEXT,
    metadata JSONB,
    PRIMARY KEY (chembl_id, source)

   
);
"""
with engine.begin() as conn:
    conn.execute(text(create_table_sql))
    #conn.commit()


OperationalError: (psycopg2.OperationalError) could not translate host name "None" to address: nodename nor servname provided, or not known

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [3]:
import pandas as pd
from sqlalchemy import create_engine, text

engine = create_engine(conn_str)

def insert_into_db(df, source_name):
    df["source"] = source_name
    df.to_sql("drug_properties", engine, if_exists="append", index=False, method="multi")

NameError: name 'conn_str' is not defined

In [ ]:
def fetch_chembl_properties(limit=1000, offset=0):
    url = "https://www.ebi.ac.uk/chembl/api/data/molecule.json"
    resp = requests.get(url, params={"limit": limit, "offset": offset})
    data = resp.json().get("molecules", [])
    df = pd.json_normalize(data)
    df = df[["molecule_chembl_id",
             "molecule_structures.canonical_smiles",
             "molecule_properties.alogp",
             "molecule_properties.cx_logp",
             "molecule_properties.full_mwt"]]
    df = df.rename(columns={
        "molecule_chembl_id": "chembl_id",
        "molecule_structures.canonical_smiles": "smiles",
        "molecule_properties.alogp": "logp",
    })
    insert_into_db(df, "CHEMBL")


def fetch_aqsol_properties():
    aqsol = pd.read_csv("AqSolDB.csv")
    df = aqsol.rename(columns={
        "Solubility": "solubility",
        "SMILES": "smiles"
    })
    insert_into_db(df, "AqSolDB")

